In [1]:
from sentence_transformers import SentenceTransformer

In [2]:
model_name = 'multi-qa-distilbert-cos-v1'
embedding_model = SentenceTransformer(model_name)

In [3]:
user_question = "I just discovered the course. Can I still join it?"

In [4]:
v = embedding_model.encode(user_question)[0]

In [5]:
v

0.07822261

In [6]:
import json

with open('documents-with-ids.json', 'rt') as f_in:
    documents = json.load(f_in)

In [7]:
filtered_docs = []

for doc in documents:
    if doc['course'] == 'machine-learning-zoomcamp':
        filtered_docs.append(doc)

In [8]:
len(filtered_docs)

375

In [9]:
from tqdm.auto import tqdm

In [10]:
X = []

for doc in tqdm(filtered_docs):
    question = doc['question']
    text = doc['text']
    qa_text = f'{question} {text}'

    embeddings = embedding_model.encode(qa_text).tolist()

    X.append(embeddings)

  0%|          | 0/375 [00:00<?, ?it/s]

In [11]:
import numpy as np

In [12]:
X = np.array(X)

In [13]:
X.shape

(375, 768)

In [14]:
v2 = embedding_model.encode(user_question)

In [15]:
scores = X.dot(v2)

In [16]:
np.max(scores)

0.6506574752353114

## Vector Search

In [17]:
class VectorSearchEngine():
    def __init__(self, documents, embeddings):
        self.documents = documents
        self.embeddings = embeddings

    def search(self, v_query, num_results=10):
        scores = self.embeddings.dot(v_query)
        idx = np.argsort(-scores)[:num_results]
        
        return [self.documents[i] for i in idx]

search_engine = VectorSearchEngine(documents=filtered_docs, embeddings=X)
search_engine.search(v2, num_results=5)

[{'text': 'Yes, you can. You won’t be able to submit some of the homeworks, but you can still take part in the course.\nIn order to get a certificate, you need to submit 2 out of 3 course projects and review 3 peers’ Projects by the deadline. It means that if you join the course at the end of November and manage to work on two projects, you will still be eligible for a certificate.',
  'section': 'General course-related questions',
  'question': 'The course has already started. Can I still join it?',
  'course': 'machine-learning-zoomcamp',
  'id': 'ee58a693'},
 {'text': 'Welcome to the course! Go to the course page (http://mlzoomcamp.com/), scroll down and start going through the course materials. Then read everything in the cohort folder for your cohort’s year.\nClick on the links and start watching the videos. Also watch office hours from previous cohorts. Go to DTC youtube channel and click on Playlists and search for {course yyyy}. ML Zoomcamp was first launched in 2021.\nOr you c

### Hit-rate for our search engine

In [18]:
import pandas as pd

In [19]:
df_ground_truth = pd.read_csv('./ground-truth-data.csv')

In [20]:
df_ground_truth.head()

,question,course,document
0,When does the course begin?,data-engineering-zoomcamp,c02e79ef
1,How can I get the course schedule?,data-engineering-zoomcamp,c02e79ef
2,What is the link for course registration?,data-engineering-zoomcamp,c02e79ef
3,How can I receive course announcements?,data-engineering-zoomcamp,c02e79ef
4,Where do I join the Slack channel?,data-engineering-zoomcamp,c02e79ef


In [21]:
df_ground_truth = df_ground_truth[df_ground_truth.course == 'machine-learning-zoomcamp']

In [22]:
df_ground_truth.head()

,question,course,document
2123,Where can I sign up for the course?,machine-learning-zoomcamp,0227b872
2124,Can you provide a link to sign up?,machine-learning-zoomcamp,0227b872
2125,Is there an FAQ for this Machine Learning course?,machine-learning-zoomcamp,0227b872
2126,Does this course have a GitHub repository for ...,machine-learning-zoomcamp,0227b872
2127,How can I structure my questions and answers f...,machine-learning-zoomcamp,0227b872


In [23]:
ground_truth = df_ground_truth.to_dict(orient='records')

In [24]:
ground_truth[:2]

[{'question': 'Where can I sign up for the course?',
  'course': 'machine-learning-zoomcamp',
  'document': '0227b872'},
 {'question': 'Can you provide a link to sign up?',
  'course': 'machine-learning-zoomcamp',
  'document': '0227b872'}]

In [25]:
def hit_rate(relevance_total):
    cnt = 0

    for line in relevance_total:
        if True in line:
            cnt = cnt + 1

    return cnt / len(relevance_total)

In [26]:
def evaluate(ground_truth, search_engine):
    relevance_total = []

    for q in tqdm(ground_truth):
        doc_id = q['document']
        question = q['question']
        
        question_vector = embedding_model.encode(question)
        
        results = search_engine.search(question_vector, num_results=5)
        
        relevance = [d['id'] == doc_id for d in results]
        
        relevance_total.append(relevance)

    return {
        'hit_rate': hit_rate(relevance_total),
    }

In [27]:
evaluate(ground_truth, search_engine)

  0%|          | 0/1830 [00:00<?, ?it/s]

{'hit_rate': 0.9398907103825137}